In [1]:
import pandas as pd
import numpy as np

In [2]:
# read all the excel files and combine in one file. give the names of domain based on filename
import os
l=os.listdir()
df_l=[]
for i in l:
    if i.startswith('7'):
        df=pd.read_excel(i,sheet_name=None)
        df=df[list(df.keys())[1]]
        name=i.split('.')[0].split('_')[0][1::]
        df['domain']=name
        df_l.append(df)

df=pd.concat(df_l)

In [10]:
ng=pd.read_csv('ngrams_list.csv')  # read the ngrams list
ng['domain']=ng['domain'].astype(str).str.lower() # make the ngram and domain columns lowercase
ng['ngram']=ng['ngram'].astype(str).str.lower()

ng=ng[(ng['ngram']!='nan')&(ng['ngram'].str.len()>1) & (~ng['ngram'].str.contains('could'))].dropna(subset=['ngram'])

ng=ng[ng['domain']!='cctr']

In [11]:
## Following function will search for the ngram in the utterance and then multiply the weight in top_p1, top_p2 and top_p3 columns if found.

def fun(row,domain):
    d={1:1.25,2:1.5,3:1.75,0:1,4:1,5:1} # multiply with 1.25 if length of ngram is 1, with 1.5 if length is 2 and 1.75 if its 3.
    l=[]
    n=[]
    try:
        ng1=ng[ng['domain']==domain].drop_duplicates(subset=['ngram'])  # select the corresponding domain in the ngram
        for i in ng1['ngram']:
            if str(i) in str(row['utterance']):
                l.append(len(str(i).split()))
                n.append(str(i))

        l1=max(l)   # take the maximum of the lengths of the ngrams so that if multiple ngrams are found prefer with largest length.
        n1=n[np.argmax(l)] # take the matched ngram
    except:
        l1=0
        n1='None'  # put none if ngram is not found in the utterance
    row['top_p1_updated']=row['top_p1']*d[l1]
    row['top_p2_updated']=row['top_p2']*d[l1]
    row['top_p3_updated']=row['top_p3']*d[l1]
    row['ngram_found']=n1
    row['domain']=domain
    return row

In [12]:
import swifter
df1=[]
for i in ng.domain.unique().tolist():
    df1.append(df.swifter.apply(lambda x: fun(x,i),axis=1))
    
df1=pd.concat(df1)

/usr/local/lib/python3.9/site-packages/swifter/swifter.py:36: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/8 [00:00<?, ?it/s]

In [17]:
df1=df1.reset_index(drop=True).reset_index()

df1=df1.drop_duplicates()

conf=df1[['index','top_p1','top_p2','top_p3']].melt(id_vars='index').rename(columns={'variable':'top_scores','value':'confidence'})

conf_updated=df1[['index','top_p1_updated','top_p2_updated','top_p3_updated']].melt(id_vars='index').rename(columns={'variable':'top_scores_updated','value':'confidence_updated'})

conf1=conf.join(conf_updated,rsuffix = '1',).drop('top_scores_updated',axis=1)

conf1['top_scores']=conf1['top_scores'].str.get(-1)

conf1.drop(['index1'],axis=1,inplace=True)

df2=pd.merge(df1,conf1,on='index',how='left')

ren={'pred_values':'intent_predicted','confidence_updated':'updated confidence','top_scores':'order of confidence scores','ngram_found':'ngram'}
df3=df2[['utterance','intent','intent1','pred_values','confidence','confidence_updated','domain','top_scores','ngram_found']].rename(columns=ren)

In [14]:
#df4=df3[df3['ngram']!='None']

In [20]:
df3.dropna(subset=['utterance'],inplace=True)
df3['utterance']=df3['utterance'].str.strip()
df3=df3[df3['utterance']!='']

In [21]:
df3.sort_values('utterance')

utterance                                             intent  \
143869    1bank delete  access_management.delete_application_account_o...   
143870    1bank delete  access_management.delete_application_account_o...   
96182     1bank delete  access_management.delete_application_account_o...   
143868    1bank delete  access_management.delete_application_account_o...   
120024    1bank delete  access_management.delete_application_account_o...   
...                ...                                                ...   
118532  zoomed font ie                            computer.zoomed_browser   
118531  zoomed font ie                            computer.zoomed_browser   
118530  zoomed font ie                            computer.zoomed_browser   
94688   zoomed font ie                            computer.zoomed_browser   
166218  zoomed font ie                            computer.zoomed_browser   

        intent1                                   intent_predicted  \
143869       22  access_management.delete_application_account_o...   
143870       22  access_management.delete_application_account_o...   
96182        22  access_management.delete_application_account_o...   
143868       22  access_management.delete_application_account_o...   
120024       22  access_management.delete_application_account_o...   
...         ...                                                ...   
118532      219                            computer.zoomed_browser   
118531      219                            computer.zoomed_browser   
118530      219                            computer.zoomed_browser   
94688       219                            computer.zoomed_browser   
166218      219                            computer.zoomed_browser   

        confidence  updated confidence   domain order of confidence scores  \
143869    0.053261            0.066577     data                          2   
143870    0.015893            0.019867     data                          3   
96182     0.015893            0.015893    easre                          3   
143868    0.890510            1.113137     data                          1   
120024    0.890510            0.890510  ibgpdts                          1   
...            ...                 ...      ...                        ...   
118532    0.002475            0.002475    easre                          3   
118531    0.003373            0.003373    easre                          2   
118530    0.976525            0.976525    easre                          1   
94688     0.002475            0.003093     iris                          3   
166218    0.976525            0.976525     data                          1   

       ngram  
143869  bank  
143870  bank  
96182   None  
143868  bank  
120024  None  
...      ...  
118532  None  
118531  None  
118530  None  
94688     ie  
166218  None  

[166845 rows x 9 columns]

In [22]:
df3.sort_values('utterance').to_excel('output.xlsx',index=None)